Imports

In [1]:
import tensorflow as tf
from PIL import ImageFile
import numpy as np
import os

ImageFile.LOAD_TRUNCATED_IMAGES = True

Process images -- code from https://www.tensorflow.org/tutorials/load_data/images

In [2]:
num_images = len(os.listdir("./images/violent")) + len(os.listdir("./images/peaceful"))
dir_images = "./images"
class_images = np.array([item for item in os.listdir(dir_images)])

image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

BATCH_SIZE = 128
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(num_images/BATCH_SIZE)

train_data_gen = image_generator.flow_from_directory(directory=str(dir_images),
                                                     subset="training",
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="binary")

validation_data_gen = image_generator.flow_from_directory(directory=str(dir_images),
                                                     subset="validation",
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode="binary")

Found 1047 images belonging to 2 classes.
Found 261 images belonging to 2 classes.


Define and train TensorFlow model

In [3]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, Dropout

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation="relu",
                 input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
model.add(Conv2D(32, kernel_size=(3,3), activation="relu"))
model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(train_data_gen,
        steps_per_epoch = 2000 // BATCH_SIZE,
        epochs = 3,
        validation_data = validation_data_gen,
        validation_steps = 800 // BATCH_SIZE)

Using TensorFlow backend.


Epoch 1/3
15/15 [==============================] - 109s 7s/step - loss: 19.7639 - accuracy: 0.5070 - val_loss: 0.6899 - val_accuracy: 0.4464
Epoch 2/3
15/15 [==============================] - 103s 7s/step - loss: 0.6965 - accuracy: 0.5076 - val_loss: 0.6873 - val_accuracy: 0.5536
Epoch 3/3
15/15 [==============================] - 105s 7s/step - loss: 0.6906 - accuracy: 0.5355 - val_loss: 0.6804 - val_accuracy: 0.5441
